In [395]:
import ccxt
import os
from dotenv import load_dotenv
import time
import pandas as pd
import numpy as np
from datetime import date, datetime, timezone, tzinfo

In [493]:
load_dotenv()

# Retrieve API keys from environment variables
xP_KEY = os.getenv('xP_KEY')
xP_SECRAT = os.getenv('xP_SECRET')

kC_KEY = os.getenv('kC_KEY')
kC_SECRET = os.getenv('kC_SECRAT')
kC_PASSWORD = os.getenv('kC_PASSWORD')


In [494]:
# Initialize the KuCoin exchange object
kucoin = ccxt.kucoin({
    'enableRateLimit': True,
    'apiKey': kC_KEY,
    'secret': kC_SECRET,
    'password': kC_PASSWORD
})
phemex = ccxt.phemex({
    'enableRateLimit': True,
    'apiKey': xP_KEY,
    'secret': xP_SECRAT
})
symbol = 'BTCUSD'
symbol_str = "STRKUSDTM"

In [495]:
def ask_bid():
    order_book = phemex.fetch_order_book(symbol=symbol)
    return order_book['asks'][0][0], order_book['bids'][0][0]

# Dialy SMA 20 Days

In [496]:
 #! FIND DAILY SMA PER 20 DAYS

def sma_20d():
    print("starting 20 Days ...")

    timeframe = '1d'
    num_bars = 100
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)
    # print(" Got the Bars: ",bars)
    data_20days = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_20days['date'] = pd.to_datetime(data_20days['timestep'], unit='ms')
    data_20days.drop(columns=['timestep'], inplace=True)
    data_20days['SMA_20'] = data_20days['close'].rolling(window=20).mean()

    bid = ask_bid()[1] 
    data_20days.loc[data_20days['SMA_20'] > bid, 'SIGNAL'] = 'SELL'
    data_20days.loc[data_20days['SMA_20'] < bid, 'SIGNAL'] = 'BUY'

    if data_20days['SIGNAL'].iloc[-1] == 'SELL':
        print("Opening as sell ...")
    else:
        print('Opening as Buy ...')
    
    return data_20days

sma_20d()


starting 20 Days ...
Opening as Buy ...


,open,high,low,close,volume,date,SMA_20,SIGNAL
0,64850.7,65048.8,63352.2,64096.5,102192438.0,2024-06-21,NaN,NaN
1,64111.9,64484.4,63919.6,64224.3,12399096.0,2024-06-22,NaN,NaN
2,64220.5,64475.0,63082.3,63133.5,34814744.0,2024-06-23,NaN,NaN
3,63139.7,63290.0,58206.9,60223.7,232487908.0,2024-06-24,NaN,NaN
4,60220.8,62372.2,60193.2,61761.2,80158692.0,2024-06-25,NaN,NaN
...,...,...,...,...,...,...,...,...
95,63412.0,64744.2,62750.0,64316.1,153646836.0,2024-09-24,59550.335,BUY
96,64316.2,64917.0,62900.0,63119.1,106151622.0,2024-09-25,59895.090,BUY
97,63126.8,65843.1,62650.0,65175.6,126242249.0,2024-09-26,60451.990,BUY
98,65175.6,66490.8,64828.6,65742.1,97440115.0,2024-09-27,61027.830,BUY


#  15 Minutes SMA

In [498]:
 #! Define the SMA for the 15 Minutes With Prices bp_1, bp_2, sp_1, sp_2

def sma_15min():
    print("starting sma 15 Minuts  ...")

    timeframe = '15m'
    num_bars = 100
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)
    # # print(" Got the Bars: ",bars)
    data_15m = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_15m['date'] = pd.to_datetime(data_15m['timestep'], unit='ms')
    data_15m.drop(columns=['timestep'], inplace=True)
    data_15m['SMA_15m'] = data_15m['close'].rolling(window=20).mean()

    bid = ask_bid()[1] 
        
    data_15m['bp_1'] = data_15m['SMA_15m'] * 1.001
    data_15m['bp_2'] = data_15m['SMA_15m'] * 0.997
    data_15m['sp_1'] = data_15m['SMA_15m'] * 0.999
    data_15m['sp_2'] = data_15m['SMA_15m'] * 1.003

    return data_15m

sma_15min()

starting sma 15 Minuts  ...


,open,high,low,close,volume,date,SMA_15m,bp_1,bp_2,sp_1,sp_2
0,65660.3,65668.4,65625.1,65625.1,96174.0,2024-09-28 15:30:00,NaN,NaN,NaN,NaN,NaN
1,65625.0,65647.9,65625.0,65626.9,17329.0,2024-09-28 15:45:00,NaN,NaN,NaN,NaN,NaN
2,65625.1,65625.1,65585.6,65609.9,123529.0,2024-09-28 16:00:00,NaN,NaN,NaN,NaN,NaN
3,65593.2,65662.5,65564.2,65660.6,409661.0,2024-09-28 16:15:00,NaN,NaN,NaN,NaN,NaN
4,65659.7,65659.7,65512.7,65528.3,195161.0,2024-09-28 16:30:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,65693.2,65800.0,65610.6,65629.9,295981.0,2024-09-29 15:15:00,65685.500,65751.185500,65488.443500,65619.814500,65882.556500
96,65629.8,65734.8,65629.8,65727.9,155584.0,2024-09-29 15:30:00,65696.525,65762.221525,65499.435425,65630.828475,65893.614575
97,65729.0,65769.2,65707.9,65729.7,172796.0,2024-09-29 15:45:00,65705.895,65771.600895,65508.777315,65640.189105,65903.012685
98,65729.9,65770.0,65713.9,65732.9,73104.0,2024-09-29 16:00:00,65710.835,65776.545835,65513.702495,65645.124165,65907.967505


# Bot

In [499]:
def bot():
    data_15m = sma_15min()
    data_20d = sma_20d()
    ask,bid = ask_bid()

    # * Define OPEN Order
    signal = data_20d.iloc[-1]['SIGNAL']
    
    open_size = 1
    
    in_pos = pnl_close()[1]


    if not in_pos:
        if signal == "BUY":
            print("Make order as BUY !!")
            bp_1 = data_15m.iloc[-1]['bp_1']
            bp_2 = data_15m.iloc[-1]['bp_2']
            print(f"bp_1: {bp_1} bp_2: {bp_2}")
            # TODO: Make Buy Order Here ...
        else:
            print("Make order as SELL !!")
            sp_1 = data_15m.iloc[-1]['sp_1']
            sp_2 = data_15m.iloc[-1]['sp_2']
            print(f"sp_1: {sp_1} sp_2: {sp_2}")
            # TODO: Make Sell Order Here ... 
    else:
        print("Already In Position, not making new Position")
bot()

starting sma 15 Minuts  ...
starting 20 Days ...
Opening as Buy ...
Checking to see if it's time to exit...
Already In Position, not making new Position


In [500]:
target = 25

def kill_switch():
    print("Starting the Kill switching ...")
    # TODO: Implement the Kill Switch

    open_pos, _, size, side = pnl_close_kucoin()
    return open_pos, size, side

def pnl_close_phemex():
    print("🔍 Checking to see if it's time to exit on Phemex...")
    
    # Example of different parameters
    params = {'type': 'future', 'currency': 'BTC'}
    poc_dic = phemex.fetch_positions(params=params)[0]
    side = poc_dic['side']
    entry_price = float(poc_dic['info']['avgEntryPrice'])
    leverage = float(poc_dic['leverage'])
    current_price = ask_bid()[1]

    if side == 'long':
        diff = current_price - entry_price
    else:
        diff = entry_price - current_price
    
    try:
        perc = ((diff / entry_price) * leverage)
    except ZeroDivisionError:
        perc = 0

    return f"📊 Difference is {diff} and Percentage: {perc:.2f}%"

def pnl_close_kucoin():
    print("Starting PNL ...")
    
    # Example of different parameters

    poc_dic = kucoin.fetch_ticker(symbol='STRK/USDT')
    perc = poc_dic['percentage']
    diff = float(poc_dic['info']['changePrice'])
    current_price = poc_dic['last']
    entry_price = float(poc_dic['open']) 
    side = 'long' if (diff >= 0 and current_price >= entry_price) else 'short'
    
    print(f"Difference: {diff}, Current Price: {current_price}, Entry Price: {entry_price}, Side: {side}, Percentage: {perc}%")


    pnlclose = False
    in_pos = False
    if perc > 0:
        # * We are in a winning position (profit)
        if perc > target:
            # ! Closing the position as target is hit
            print("In Profit also Hit the Target ...")
            pnlclose = True
            kill_switch()
        else:
            print("In Profit but  not hit the target, so wait !!")
    else:
        in_pos = True
        print("We are not in Profit yet.")
    print("Finishing the PNL close ...")

    return pnlclose, in_pos,"size",side

# print(pnl_close_phemex())
# pnl_close_kucoin()
kill_switch()


Starting the Kill switching ...
Starting PNL ...
Difference: 0.008, Current Price: 0.47283, Entry Price: 0.46483, Side: long, Percentage: 1.72%
In Profit but  not hit the target, so wait !!
Finishing the PNL close ...


(False, 'size', 'long')

In [503]:
import json

from kucoin_futures.client import Trade

client = Trade(key=kC_KEY, secret=kC_SECRET, passphrase=kC_PASSWORD)

positions = client.get_position_details(symbol="STRKUSDTM")

# Write positions to sample.json
with open('../sample.json', 'w') as f:
    json.dump(positions, f, indent=4)

positions

{'id': '300000000001847863',
 'symbol': 'STRKUSDTM',
 'autoDeposit': False,
 'crossMode': False,
 'maintMarginReq': 0.012,
 'riskLimit': 10000,
 'realLeverage': 1.0,
 'delevPercentage': 0.32,
 'openingTimestamp': 1727612017604,
 'currentTimestamp': 1727627637703,
 'currentQty': 2,
 'currentCost': 0.9278,
 'currentComm': 0.00055668,
 'unrealisedCost': 0.9278,
 'realisedGrossCost': 0.0,
 'realisedCost': 0.00055668,
 'isOpen': True,
 'markPrice': 0.4734,
 'markValue': 0.9468,
 'posCost': 0.9278,
 'posCross': 0,
 'posCrossMargin': 0,
 'posInit': 0.9278,
 'posComm': 0.00111336,
 'posCommCommon': 0.00111336,
 'posLoss': 0,
 'posMargin': 0.92891336,
 'posFunding': 0,
 'posMaint': 0.01224696,
 'maintMargin': 0.94791336,
 'realisedGrossPnl': 0.0,
 'realisedPnl': -0.00055668,
 'unrealisedPnl': 0.019,
 'unrealisedPnlPcnt': 0.0205,
 'unrealisedRoePcnt': 0.0205,
 'avgEntryPrice': 0.4639,
 'liquidationPrice': 0.0056,
 'bankruptPrice': 0.0001,
 'settleCurrency': 'USDT',
 'maintainMargin': 0.012,
 'ri